# Step 2: Detecting Peaks

---

Peak detection is a common problem in time-series analysis. In some cases, they are very easy to spot by eye, but that can be difficult to define mathematically. This is particularly true for signals that are “noisy” or have pronounced variations in their baseline values. There are several Python libraries out there for automatically identifying peaks in time-series data, such as [`findpeaks.py`](https://erdogant.github.io/findpeaks/pages/html/index.html) and [`PeakUtils`](https://peakutils.readthedocs.io/en/latest/). In `hplc-py`, peak detection is executed using the [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.html) which is very mature and actively maintained. In this notebook, we won’t cover the algorithms used under-the-hood for peak detection, but will outline how `hplc-py` leverages `scipy.signal.find_peaks` and `scipy.signal.peak_widths` to 1) identify peaks in chromatographic data and 2) clip the chromatogram into discrete peak windows which are used in the fitting procedure.

## What's in a peak?